Data sources from https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
Excercise in producing a decision tree to predict forest cover of parcels of land based on the 50+ tributes.
Based on the case study in Advnaced Analytics using Spark
Using Almond Shell kernel for Jupyter notebook

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.3/spark-sql_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.8.2/almond-spark_2.12-0.8.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.3/spark-parent_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.3/spark-catalyst_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/spark-project/spark/unused/1.0.0/unused-1.0.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/xbean/xbean-asm6-shaded/4.8/xbean-asm6-shaded-4.8.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammonite-spark_2.12/0.7.2/ammonite-spark_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-vector/0.10.0/arrow-vector-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-colum

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.3/spark-network-shuffle_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/joda-time/joda-time/2.9.9/joda-time-2.9.9.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.1.0/scala-parser-combinators_2.12-1.1.0.pom
Downloaded https://repo1.maven.org/maven2/com/vlkan/flatbuffers/1.2.0-3f79e055/flatbuffers-1.2.0-3f79e055.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-core-asl/1.9.13/jackson-core-asl-1.9.13.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-json/3.1.5/metrics-json-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-common_2.12/2.4.3/spark-network-common_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.6.7.1/jackson-module-scala_2.12-2.6.7.1.pom
Downloaded https://repo1.maven.org/maven2

Downloaded https://repo1.maven.org/maven2/com/google/code/findbugs/jsr305/3.0.2/jsr305-3.0.2.pom
Downloaded https://repo1.maven.org/maven2/com/github/luben/zstd-jni/1.3.2-2/zstd-jni-1.3.2-2.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/jul-to-slf4j/1.7.16/jul-to-slf4j-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/commons-codec/commons-codec/1.10/commons-codec-1.10.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.7/antlr4-runtime-4.7.pom
Downloaded https://repo1.maven.org/maven2/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0.pom
Downloaded https://repo1.maven.org/maven2/com/twitter/chill_2.12/0.9.3/chill_2.12-0.9.3.pom
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-graphite/3.1.5/metrics-graphite-3.1.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-format/0.10.0/arrow-format-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.pom
Dow

Downloaded https://repo1.maven.org/maven2/org/apache/apache/7/apache-7.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.25/slf4j-parent-1.7.25.pom
Downloaded https://repo1.maven.org/maven2/com/carrotsearch/hppc-parent/0.7.2/hppc-parent-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/41/commons-parent-41.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer-parent/2.8/paranamer-parent-2.8.pom
Downloaded https://repo1.maven.org/maven2/io/airlift/airbase/78/airbase-78.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/project/2.22.2/project-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.6.1/jackson-parent-2.6.1.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.16/slf4j-parent-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/40/commons-parent-40.pom
Downloaded https://repo1.maven.

Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-core_2.12/3.5.3/json4s-core_2.12-3.5.3.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-io/9.4.20.v20190813/jetty-io-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/media/jersey-media-jaxb/2.22.2/jersey-media-jaxb-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-hdfs/2.6.5/hadoop-hdfs-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.6.7/jackson-annotations-2.6.7.pom
Downloaded https://repo1.maven.org/maven2/javax/annotation/javax.annotation-api/1.2/javax.annotation-api-1.2.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bundles/repackaged/jersey-guava/2.22.2/jersey-guava-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/hk2-api/2.4.0-b34/hk2-api-2.4.0-b34.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.8/scala-reflect-2.

...

Downloaded https://repo1.maven.org/maven2/xerces/xercesImpl/2.9.1/xercesImpl-2.9.1-sources.jar
Downloaded https://repo1.maven.org/maven2/xml-apis/xml-apis/1.3.04/xml-apis-1.3.04-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-api/2.6.5/hadoop-yarn-api-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/xml/stream/stax-api/1.0-2/stax-api-1.0-2-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/activation/activation/1.1.1/activation-1.1.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/2.6.5/hadoop-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/xml/bind/jaxb-api/2.2.2/jaxb-api-2.2.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-jaxrs/1.9.13/jackson-jaxrs-1.9.13-sources.jar
Downloaded https://repo1.maven.org/maven2/org/codehaus/jackson/jackson-xc/1.9.13/jackson-xc-1.9.13-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/i

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.3/spark-unsafe_2.12-2.4.3-sources.jar
Downloaded https://repo1.maven.org/maven2/javax/servlet/javax.servlet-api/3.1.0/javax.servlet-api-3.1.0-sources.jar
Downloaded https://repo1.maven.org/maven2/com/ning/compress-lzf/1.0.3/compress-lzf-1.0.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/lz4/lz4-java/1.4.0/lz4-java-1.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/com/github/luben/zstd-jni/1.3.2-2/zstd-jni-1.3.2-2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/slf4j/jcl-over-slf4j/1.7.16/jcl-over-slf4j-1.7.16-sources.jar
Downloaded https://repo1.maven.org/maven2/io/netty/netty-all/4.1.17.Final/netty-all-4.1.17.Final-sources.jar
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-scalap_2.12/3.5.3/json4s-scalap_2.12-3.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.7.45/RoaringBitmap-0.7.45-sources.jar
Downloaded https://re

Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-parser-combinators_2.12/1.1.0/scala-parser-combinators_2.12-1.1.0-sources.jar
Downloaded https://repo1.maven.org/maven2/net/sf/py4j/py4j/0.10.7/py4j-0.10.7-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/ivy/ivy/2.4.0/ivy-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/commons-compiler/3.0.9/commons-compiler-3.0.9-sources.jar
Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/janino/3.0.9/janino-3.0.9-sources.jar
Downloaded https://repo1.maven.org/maven2/io/airlift/aircompressor/0.10/aircompressor-0.10-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-shims/1.5.5/orc-shims-1.5.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/2.4.3/spark-core_2.12-2.4.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-core/1.5.5/orc-core-1.5.5-sources.jar
Downloaded https://repo1.maven.org

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.3/spark-sql_2.12-2.4.3-sources.jar
Downloaded https://repo1.maven.org/maven2/xmlenc/xmlenc/0.52/xmlenc-0.52-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/xerces/xercesImpl/2.9.1/xercesImpl-2.9.1-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/mortbay/jetty/jetty-util/6.1.26/jetty-util-6.1.26.jar
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.6.7.1/jackson-databind-2.6.7.1.jar


import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}

import org.apache.log4j.{Logger,Level}

In [3]:
Logger.getLogger("org").setLevel(Level.ERROR)

Create the sql sessions 

In [4]:
val spark = {
            org.apache.spark.sql.NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2-sources.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.jar


Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@17fe980f

In [5]:
val data = spark.read
            .option("header","false")
            .option("inferSchema","true")
            .csv("../covtype/")

csv at cmd4.sc:4

1 / 1

csv at cmd4.sc:4

4 / 4

data: DataFrame = [_c0: string, _c1: string ... 53 more fields]

In [6]:
data.count()

count at cmd5.sc:1

4 / 4

count at cmd5.sc:1

1 / 1

res5: Long = 581259L

In [7]:
data.head()

head at cmd6.sc:1

1 / 1

res6: Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]

In [8]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _c27

Looks like the schema isn't what was expected. _c0..2 should be integers

In [9]:
import org.apache.spark.sql.types._

import org.apache.spark.sql.types._

Have to update the scheme. Only way seems to be to create new DF every time.

In [10]:
val data2=data.withColumn("_c0",data("_c0").cast(IntegerType))
val data3=data2.withColumn("_c1",data2("_c1").cast(IntegerType))
val data4=data3.withColumn("_c2",data3("_c2").cast(IntegerType))

data2: DataFrame = [_c0: int, _c1: string ... 53 more fields]
data3: DataFrame = [_c0: int, _c1: int ... 53 more fields]
data4: DataFrame = [_c0: int, _c1: int ... 53 more fields]

In [11]:
data4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

Above approach worked by duplicating the dataframe everytime seem thereis potential for memory overflow. Try doing it when adding titles.

Create header titles for the columns. 

In [12]:
val colNames = Seq(
"Elevation", "Aspect", "Slope",
"Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology",
"Horizontal_Distance_To_Roadways",
"Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm",
"Horizontal_Distance_To_Fire_Points"
) ++ (
(0 until 4).map(i => s"Wilderness_Area_$i")
) ++ (
(0 until 40).map(i => s"Soil_Type_$i")
) ++ Seq("Cover_Type")

colNames: Seq[String] = List(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...

In [13]:
colNames.getClass

res12: Class[T] = class scala.collection.immutable.$colon$colon

implicits contains the function $ which returns a column object. Needed to access the data type in order to cast it to integer. $("string"): column   ... pretty 'hacky' syntax.

In [242]:
import spark.implicits._

import spark.implicits._

In [15]:
val dataClean = data.toDF(colNames:_*).
    withColumn("Elevation",$"Elevation".cast(IntegerType))
    .withColumn("Aspect",$"Aspect".cast(IntegerType))
    .withColumn("Slope",$"Slope".cast(IntegerType))
    .withColumn("Cover_Type",$"Cover_Type".cast(DoubleType))


dataClean: DataFrame = [Elevation: int, Aspect: int ... 53 more fields]

In [16]:
dataClean.show() // Using the spark.sql API

show at cmd15.sc:1

1 / 1

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+-----------------+-----------------+-----------------+-----------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness

|     2617|    45|    9|                             240|                            56|                            666|          223|           221|          133|                              6244|                1|                0|                0|                0|          0|          0|          0|          0|          0|          0|          0|          0|          0|          0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           1|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|           0|       5.0|
|     2612|    59|   10|                             247|                            11|                            636|          228|           219|          124|                              6230|          

Randomly split the data into train and test using 10% for testing (will need to further split the training data if CV performed

In [102]:
val Array(trainData, testData) = dataClean.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res101_1: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]
res101_2: Dataset[Row] = [Elevation: int, Aspect: int ... 53 more fields]

Vectorise the features. This is required by all SparkML alogorithms.

Took some messing around to find the right repo for ml lib. Eventually found a reference on the Mvnrepository site to find the mllib version that goes with scala 2.12.

In [18]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.12/2.4.0/spark-mllib_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.0/spark-parent_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-graphx_2.12/2.4.0/spark-graphx_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.12/0.13.2/breeze_2.12-0.13.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib-local_2.12/2.4.0/spark-mllib-local_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-streaming_2.12/2.4.0/spark-streaming_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.0/spark-tags_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.0/spark-sql_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/2.4.0/spark-core_2.12-2.4.0.pom
Downloaded http

Downloaded https://repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer-parent/2.7/paranamer-parent-2.7.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/codehaus-parent/4/codehaus-parent-4.pom
Downloaded https://repo1.maven.org/maven2/commons-beanutils/commons-beanutils-core/1.8.0/commons-beanutils-core-1.8.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.0/spark-tags_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.0/spark-network-shuffle_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.0/spark-unsafe_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/commons-beanutils/commons-beanutils-core/1.8.0/commons-beanutils-core-1.8.0.jar
Downloaded https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.5.11/RoaringBitmap-0.5.11.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-kvstore_2.12/2.4.0/spark-kvs

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-launcher_2.12/2.4.0/spark-launcher_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.12/2.4.0/spark-mllib_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-kvstore_2.12/2.4.0/spark-kvstore_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.0/spark-network-shuffle_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.12/0.13.0/spire_2.12-0.13.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.0/spark-unsafe_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.5.11/RoaringBitmap-0.5.11-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-common_2.12/2.4.0/spark-network-common_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/

import $ivy.$                                    

In [19]:
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.feature.VectorAssembler

In [103]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new org.apache.spark.ml.feature.VectorAssembler()
                    .setInputCols(inputCols)
                    .setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_5c9b25a78f15

Since spark 2.4 vector assembler extends HadHandleInvalid which requires you to specify how nulls will be handled. It take a single parameter ("skip") to skip when encountered. If not used then the dataset must be cleaned up and the missing data imputed. 

See stackoverflow
https://stackoverflow.com/questions/41362295/sparkexception-values-to-assemble-cannot-be-null


TODO: Investigate the source of nulls

In [104]:
val assembledTrainData = assembler.setHandleInvalid("skip").transform(trainData)

assembledTrainData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

In [105]:
assembledTrainData.select("featureVector").show(truncate=false)

show at cmd104.sc:1

1 / 1

+-----------------------------------------------------------------------------------------------------+
|featureVector                                                                                        |
+-----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0])  |
|(54,[0,1,2,5,6,7,8,9,13,18],[1874.0,18.0,14.0,90.0,208.0,209.0,135.0,793.0,1.0,1.0])                 |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1879.0,28.0,19.0,30.0,12.0,95.0,209.0,196.0,117.0,778.0,1.0,1.0])   |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1888.0,33.0,22.0,150.0,46.0,108.0,209.0,185.0,103.0,735.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1889.0,353.0,30.0,95.0,39.0,67.0,153.0,172.0,146.0,600.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,18],[1896.0,337.0,12.0,30.0,6.0,175.0,195.0,224.0,168.0,732.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1898.0,34.0,23.0,175.0,56.0,13

In [23]:
assembledTrainData.getClass

res22: Class[T] = class org.apache.spark.sql.Dataset

In [24]:
trainData.getClass

res23: Class[T] = class org.apache.spark.sql.Dataset

Note on above. The transform fucntion converts a dataframe to a dataframe but re-organises it.

Now to create the DecisionTreem classifier for predicting the group cover type (a categorical valiable ranging 1..7)

In [25]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import scala.util.Random

import org.apache.spark.ml.classification.DecisionTreeClassifier

import scala.util.Random

In [106]:
val classifier = new org.apache.spark.ml.classification.DecisionTreeClassifier()
                        .setSeed(Random.nextLong())
                        .setLabelCol("Cover_Type")
                        .setFeaturesCol("featureVector")
                        .setPredictionCol("prediction")


classifier: DecisionTreeClassifier = dtc_e68c7fc9cdf6

In [107]:
val model = classifier.fit(assembledTrainData)
//print(model.toDebugString)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

4 / 4

collectAsMap at RandomForest.scala:557

4 / 4

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_e68c7fc9cdf6) of depth 5 with 45 nodes

In [108]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_e68c7fc9cdf6) of depth 5 with 45 nodes
  If (feature 0 <= 3051.5)
   If (feature 0 <= 2569.5)
    If (feature 10 <= 0.5)
     If (feature 0 <= 2467.5)
      If (feature 3 <= 15.0)
       Predict: 4.0
      Else (feature 3 > 15.0)
       Predict: 3.0
     Else (feature 0 > 2467.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
    Else (feature 10 > 0.5)
     If (feature 9 <= 5379.0)
      Predict: 2.0
     Else (feature 9 > 5379.0)
      If (feature 5 <= 569.5)
       Predict: 2.0
      Else (feature 5 > 569.5)
       Predict: 5.0
   Else (feature 0 > 2569.5)
    If (feature 0 <= 2959.5)
     If (feature 15 <= 0.5)
      If (feature 17 <= 0.5)
       Predict: 2.0
      Else (feature 17 > 0.5)
       Predict: 3.0
     Else (feature 15 > 0.5)
      Predict: 3.0
    Else (feature 0 > 2959.5)
     If (feature 3 <= 191.0)
      If (feature 36 <= 0.5)
       Predict: 2.0
      Else (feature 36 > 0.5)

Cool! Above cell appears to show a recursive definition (and almost self explaining) of the decision tree model.
So now the model has been fit (estimator(ed) in spark speak) we can use the model to make predictions on the test data. 
Note that the default settings have been used on the above model so we don't expect this out of the box model to do very well.

Without getting under the hood I assume that the decison tree optimises the gini index for the tree. TODO find out.

DecisiontTreeClassifiers have an 'importance' dunction to assess the importance of each feature. Not obvious what this is but assume its some sort of sensitivity analysis.??

In [109]:
model.featureImportances.toArray.zip(inputCols).sorted.reverse.foreach(println)

(0.7984213228005778,Elevation)
(0.03694030855112407,Horizontal_Distance_To_Hydrology)
(0.03542484320930261,Wilderness_Area_0)
(0.029107477693662023,Soil_Type_3)
(0.028360685527603097,Soil_Type_31)
(0.02552752122536986,Hillshade_Noon)
(0.02244706813601064,Soil_Type_1)
(0.011497589048750976,Wilderness_Area_2)
(0.006088935259228888,Horizontal_Distance_To_Roadways)
(0.005744023838478082,Soil_Type_22)
(4.4022470989191995E-4,Horizontal_Distance_To_Fire_Points)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_9)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_21)
(0.0,Soil_Type_20)
(0.0,S

So the above model has the elevation as by far and away the most important feature. However we have not assessed the performance of modey yet so this should be treated cautiously until this is done. 

Note that the decisionTreeClassificationModel is a transformed ( in spark speak this is just a function that transform a dataframe into a dataframe

In [110]:
val preds = model.transform(assembledTrainData)

preds.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd109.sc:3

1 / 1

+----------+----------+------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                       |
+----------+----------+------------------------------------------------------------------------------------------------------------------+
|6.0       |3.0       |[0.0,0.0,0.03981638234145071,0.6256022919650996,0.047336892824586536,0.0,0.28724443286886314,0.0]                 |
|6.0       |4.0       |[0.0,4.1186161449752884E-4,0.051070840197693576,0.28171334431630973,0.4085667215815486,0.0,0.2582372322899506,0.0]|
|6.0       |3.0       |[0.0,0.0,0.03981638234145071,0.6256022919650996,0.047336892824586536,0.0,0.28724443286886314,0.0]                 |
|6.0       |3.0       |[0.0,0.0,0.03981638234145071,0.6256022919650996,0.047336892824586536,0.0,0.28724443286886314,0.0]                 |
|6.0       |3.0       |[0.0

preds: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [31]:
preds.columns.foreach(println)

Elevation
Aspect
Slope
Horizontal_Distance_To_Hydrology
Vertical_Distance_To_Hydrology
Horizontal_Distance_To_Roadways
Hillshade_9am
Hillshade_Noon
Hillshade_3pm
Horizontal_Distance_To_Fire_Points
Wilderness_Area_0
Wilderness_Area_1
Wilderness_Area_2
Wilderness_Area_3
Soil_Type_0
Soil_Type_1
Soil_Type_2
Soil_Type_3
Soil_Type_4
Soil_Type_5
Soil_Type_6
Soil_Type_7
Soil_Type_8
Soil_Type_9
Soil_Type_10
Soil_Type_11
Soil_Type_12
Soil_Type_13
Soil_Type_14
Soil_Type_15
Soil_Type_16
Soil_Type_17
Soil_Type_18
Soil_Type_19
Soil_Type_20
Soil_Type_21
Soil_Type_22
Soil_Type_23
Soil_Type_24
Soil_Type_25
Soil_Type_26
Soil_Type_27
Soil_Type_28
Soil_Type_29
Soil_Type_30
Soil_Type_31
Soil_Type_32
Soil_Type_33
Soil_Type_34
Soil_Type_35
Soil_Type_36
Soil_Type_37
Soil_Type_38
Soil_Type_39
Cover_Type
featureVector
rawPrediction
probability
prediction


In [111]:
val results = preds.select("Cover_Type","prediction")

results: DataFrame = [Cover_Type: double, prediction: double]

In [112]:
results.selectExpr("Cover_Type==prediction as Matched","Cover_Type")
    .groupBy("Cover_Type").pivot("Matched",Seq("true","false")).count().show()

show at cmd111.sc:2

4 / 4

show at cmd111.sc:2

200 / 200

show at cmd111.sc:2

...

show at cmd111.sc:2

100 / 100

show at cmd111.sc:2

75 / 75

+----------+------+-----+
|Cover_Type|  true|false|
+----------+------+-----+
|       7.0|  6440|12000|
|       1.0|125558|65008|
|       4.0|   992| 1472|
|       3.0| 26265| 5969|
|       2.0|206961|47882|
|       6.0|  null|15602|
|       5.0|    87| 8439|
+----------+------+-----+



In [113]:
val predPivot = results.selectExpr("Cover_Type","prediction")
    .groupBy("Cover_Type")
    .pivot("prediction",(1 to 7))
    .count()
    .na.fill(0).sort("Cover_Type")

predPivot: Dataset[Row] = [Cover_Type: double, 1: bigint ... 6 more fields]

In [114]:
predPivot.show()

show at cmd113.sc:1

4 / 4

show at cmd113.sc:1

200 / 200

show at cmd113.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|125558| 63369|  163|  1|  0|  0|1475|
|       2.0| 42993|206961| 4523|124| 76|  0| 166|
|       3.0|     0|  5285|26265|684|  0|  0|   0|
|       4.0|     0|    18| 1454|992|  0|  0|   0|
|       5.0|     0|  7786|  653|  0| 87|  0|   0|
|       6.0|     0|  5504| 9471|627|  0|  0|   0|
|       7.0| 11726|   214|   60|  0|  0|  0|6440|
+----------+------+------+-----+---+---+---+----+



TODO: Add row total

In [115]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

In [116]:
val evaluator = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator()
                .setLabelCol("Cover_Type")
                .setPredictionCol("prediction")

evaluator: MulticlassClassificationEvaluator = mcEval_945875ebcbc1

In [117]:
evaluator.setMetricName("accuracy").evaluate(preds)

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res116: Double = 0.7008236475821495

As expected accuracy not great. Also, looking at the prediction pivot table shows that there is class imbalance that might be affecting the utility of accuracy and f1 might be a better score.


In [39]:
evaluator.setMetricName("f1").evaluate(preds)

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

map at MulticlassMetrics.scala:45

...

map at MulticlassMetrics.scala:50

4 / 4

collectAsMap at MulticlassMetrics.scala:52

4 / 4

res38: Double = 0.6803906494886994

Actually not that difference suggesting that the algorithm (with the current set of parameters) isn't greatly affected by the class imbalance. This should be monitored on going as the imbalance of the data warrants this.

Its seems that spark has a confusion matrix ( what I produced in the above pivot table).Its accesses (in the book at least) via the Mllib rdd API (which is in maintenance mode now with the dataframe spark.ml api being the promoted one)

In [118]:
import org.apache.spark.mllib.evaluation.MulticlassMetrics
val predictionRDD = preds.
        select("prediction","Cover_Type").
        as[(Double,Double)].
        rdd
val multiclassMetrics = new org.apache.spark.mllib.evaluation.MulticlassMetrics(predictionRDD)

import org.apache.spark.mllib.evaluation.MulticlassMetrics

predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[2323] at rdd at cmd117.sc:4
multiclassMetrics: MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@5130238c

In [41]:
multiclassMetrics.confusionMatrix

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

map at MulticlassMetrics.scala:55

4 / 4

collectAsMap at MulticlassMetrics.scala:57

4 / 4

res40: org.apache.spark.mllib.linalg.Matrix = 121711.0  63690.0   171.0    1.0     0.0  0.0  5128.0   
43191.0   206578.0  4415.0   132.0   0.0  0.0  733.0    
0.0       5245.0    26279.0  685.0   0.0  0.0  0.0      
0.0       16.0      1462.0   1002.0  0.0  0.0  0.0      
0.0       7971.0    589.0    0.0     0.0  0.0  0.0      
0.0       5435.0    9516.0   635.0   0.0  0.0  0.0      
8031.0    219.0     35.0     0.0     0.0  0.0  10229.0  

In [119]:
predPivot.show()

show at cmd118.sc:1

4 / 4

show at cmd118.sc:1

200 / 200

show at cmd118.sc:1

200 / 200

+----------+------+------+-----+---+---+---+----+
|Cover_Type|     1|     2|    3|  4|  5|  6|   7|
+----------+------+------+-----+---+---+---+----+
|       1.0|125558| 63369|  163|  1|  0|  0|1475|
|       2.0| 42993|206961| 4523|124| 76|  0| 166|
|       3.0|     0|  5285|26265|684|  0|  0|   0|
|       4.0|     0|    18| 1454|992|  0|  0|   0|
|       5.0|     0|  7786|  653|  0| 87|  0|   0|
|       6.0|     0|  5504| 9471|627|  0|  0|   0|
|       7.0| 11726|   214|   60|  0|  0|  0|6440|
+----------+------+------+-----+---+---+---+----+



To evaluate the 70% accuracy from the default model we can benchmark it against a null model and use an inner product of the class probabilities to compare.

In [120]:
def classProbs(df: DataFrame):Array[Double] = {
    val totalCounts = df.count().asInstanceOf[Double]
        df.groupBy("Cover_Type").count()
        .orderBy("Cover_Type")
        .select("count").as[Double]
        .map(_/totalCounts)
        .collect()
}

defined function classProbs

Take inner product of test and train class probabilities as A measure of similarity

In [121]:
val trainProbs = classProbs(trainData)
val testProbs = classProbs(testData)
trainProbs.zip(testProbs).map(x=>x._1*x._2).sum

count at cmd119.sc:2

4 / 4

count at cmd119.sc:2

1 / 1

collect at cmd119.sc:6

...

collect at cmd119.sc:6

200 / 200

collect at cmd119.sc:6

8 / 8

trainProbs: Array[Double] = Array(
  4.226461858572259E-4,
  0.3644434074844711,
  0.48736842507879197,
  0.0616451454973838,
  0.004712218108381017,
  0.016305345613659313,
  0.02983767326581194,
  0.03526513876564365
)
testProbs: Array[Double] = Array(
  4.454877233863921E-4,
  0.3645117625893117,
  0.4876034473896133,
  0.06031218408923462,
  0.004848962527628806,
  0.01656871648133235,
  0.030241762760653153,
  0.035467676438839676
)
res120_2: Double = 0.3766506990757954

The null model of randomly choosing a cover type label and using it as the prediction yields a 37% true positive rate. Makes the 70% TP rate look pretty good. 

## PIPELINES

In [122]:
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.Pipeline

In [123]:
val inputCols = trainData.columns.filter(_!="Cover_Type")
val assembler = new VectorAssembler().
                    setHandleInvalid("skip").
                    setInputCols(inputCols).
                    setOutputCol("featureVector")

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "Wilderness_Area_0",
  "Wilderness_Area_1",
  "Wilderness_Area_2",
  "Wilderness_Area_3",
  "Soil_Type_0",
  "Soil_Type_1",
  "Soil_Type_2",
  "Soil_Type_3",
  "Soil_Type_4",
  "Soil_Type_5",
  "Soil_Type_6",
  "Soil_Type_7",
  "Soil_Type_8",
  "Soil_Type_9",
  "Soil_Type_10",
  "Soil_Type_11",
  "Soil_Type_12",
  "Soil_Type_13",
  "Soil_Type_14",
  "Soil_Type_15",
  "Soil_Type_16",
  "Soil_Type_17",
  "Soil_Type_18",
  "Soil_Type_19",
  "Soil_Type_20",
  "Soil_Type_21",
  "Soil_Type_22",
  "Soil_Type_23",
...
assembler: VectorAssembler = vecAssembler_fa66bdefc62f

In [124]:
val classifer = new DecisionTreeClassifier().
                setSeed(Random.nextLong()).
                setLabelCol("Cover_Type").
                setFeaturesCol("featureVector").
                setPredictionCol("prediction")

classifer: DecisionTreeClassifier = dtc_a2b444c9f0a7

The above two transformations, recall that a transformation is a map from one dataframe to another that is part of the DAG, are chained together in a pipeline that is a convenience for defining a series of transforms on the data. We do this to streamline the hyper parameter tuning process.

In [125]:
val pipeline = new Pipeline().setStages(Array(assembler,classifier))

pipeline: Pipeline = pipeline_8fa9b0202d61

Use spark.ml.tuning.ParameterBuilder to set up collections of hyper parameters to try. Will obviously need an evaluation metric as well. The ParamGrid BUilder defines the series of values to trye, and the loss function

In [126]:
import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

import org.apache.spark.ml.tuning.{ParamGridBuilder,TrainValidationSplit}

In [127]:
val paramGrid = new ParamGridBuilder().
                    addGrid(classifer.impurity, Seq("gini","entropy")).
                    addGrid(classifier.maxDepth, Seq(1,20)).
                    addGrid(classifier.maxBins,Seq(40,300)).
                    addGrid(classifier.minInfoGain, Seq(0.0,0.05)).
                    build()



paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	dtc_a2b444c9f0a7-impurity: gini,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 1,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
},
  {
	dtc_a2b444c9f0a7-impurity: entropy,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 1,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
},
  {
	dtc_a2b444c9f0a7-impurity: gini,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 1,
	dtc_e68c7fc9cdf6-minInfoGain: 0.05
},
  {
	dtc_a2b444c9f0a7-impurity: entropy,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 1,
	dtc_e68c7fc9cdf6-minInfoGain: 0.05
},
  {
	dtc_a2b444c9f0a7-impurity: gini,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
},
  {
	dtc_a2b444c9f0a7-impurity: entropy,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
},
  {
	dtc_a2b444c9f0a7-impurity: gini,
...

In [128]:
val multiclassEval = new MulticlassClassificationEvaluator().
setLabelCol("Cover_Type").
setPredictionCol("prediction").
setMetricName("accuracy")

multiclassEval: MulticlassClassificationEvaluator = mcEval_1d71318dca31

In [129]:
val validator = new TrainValidationSplit().
    setSeed(Random.nextLong()).
    setEstimator(pipeline).
    setEvaluator(multiclassEval).
    setEstimatorParamMaps(paramGrid).
    setTrainRatio(0.9)

validator: TrainValidationSplit = tvs_643091abb88b

Now go ahead and fit the model on all the parameters set in the setEstimatorParamMaps

In [130]:
val validatorModel=validator.fit(trainData)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

mapPartitions at RandomForest.scala:538

4 / 4

collectAsMap at RandomForest.scala:557

4 / 4

validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_643091abb88b

The validatorModel now contains the best model parameters for the pipeline supplied (in the setEstimator - recall that estimator is one of the spark.ml 'primatives' along with transform and is used for fitting parameters). Because a pipeline was supplied to the estimator the Model returned is wrapped in a pipeline object so we need to get the model out of the pipeline.

In [131]:
import org.apache.spark.ml.PipelineModel

import org.apache.spark.ml.PipelineModel

In [132]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res131: org.apache.spark.ml.param.ParamMap = {
	dtc_e68c7fc9cdf6-cacheNodeIds: false,
	dtc_e68c7fc9cdf6-checkpointInterval: 10,
	dtc_e68c7fc9cdf6-featuresCol: featureVector,
	dtc_e68c7fc9cdf6-impurity: gini,
	dtc_e68c7fc9cdf6-labelCol: Cover_Type,
	dtc_e68c7fc9cdf6-maxBins: 300,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-maxMemoryInMB: 256,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0,
	dtc_e68c7fc9cdf6-minInstancesPerNode: 1,
	dtc_e68c7fc9cdf6-predictionCol: prediction,
	dtc_e68c7fc9cdf6-probabilityCol: probability,
	dtc_e68c7fc9cdf6-rawPredictionCol: rawPrediction,
	dtc_e68c7fc9cdf6-seed: -2190394278285637533
}

Its interesting that the results obtained are very different to the book. This model uses 300 bins whereas the book used only the 40. Gini index was the one that performed the best compared to the entropy that the book found. 0.0 infomration gain threshold was the same (and suggests over fitting).

To get the model out of the pipeline use the code below. note that the code val best= validatorModel.bestModel throws an error. Not clear on why but the code below returns as class with the suffix 'Model' which in spark speak is the trianed model. This should then be useable for making predictions on the test data.

In [133]:
val best = validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last

best: org.apache.spark.ml.Transformer = DecisionTreeClassificationModel (uid=dtc_e68c7fc9cdf6) of depth 20 with 27393 nodes

In [134]:
val paramsAndMetrics = validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

paramsAndMetrics: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.9046406555117357,
    {
	dtc_a2b444c9f0a7-impurity: gini,
	dtc_e68c7fc9cdf6-maxBins: 300,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
}
  ),
  (
    0.9046406555117357,
    {
	dtc_a2b444c9f0a7-impurity: entropy,
	dtc_e68c7fc9cdf6-maxBins: 300,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
}
  ),
  (
    0.9040280277214076,
    {
	dtc_a2b444c9f0a7-impurity: gini,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
}
  ),
  (
    0.9040280277214076,
    {
	dtc_a2b444c9f0a7-impurity: entropy,
	dtc_e68c7fc9cdf6-maxBins: 40,
	dtc_e68c7fc9cdf6-maxDepth: 20,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0
}
  ),
  (
    0.6720526859899683,
...

The above .validationMetrics method shows that the top performing parameters result in increbilble close accuarcy scores so the results here are not so different to those in the book.The lesser work 4 bins might end up being a best choice. 

In [58]:
multiclassEval.evaluate(best.transform(testData))

: 

In [135]:
val assembledTestData = assembler.setHandleInvalid("skip").transform(testData)

assembledTestData: DataFrame = [Elevation: int, Aspect: int ... 54 more fields]

Note: because I extracted a model from the pipelineModel returned by the trainvalidationsplit training on the parameter grid then I need to assemble the data into a vector. If it was a pipelineModel ( a full pipeline returend from the training) then this would have been handled by the assembler stage in the pipeline I build.

In [136]:
multiclassEval.evaluate(best.transform(assembledTestData))

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res135: Double = 0.9052231002622693

The accuracy went up to 90%!

In [137]:
multiclassEval.evaluate(best.transform(assembledTrainData))

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res136: Double = 0.9408963505046157

The above analysis used one hot encoding of the categorical vectors. One hot encoding treats each possible value of the categorical variable as an independant feature that can take values between 0 and 1 even though these are the only two values possilble. The description in the book suggests that there is information inefficiency in doing this and that registering them as categorical variables does improve accuracy. One explaination might be the reduction in the dimension of the problem if one does not one-hot encode things. The process of registering as a categorical variable is to replace the one hot encoded data with a column whose value is a vector of length equal to the number of categories PLUS using VectorIndexer to register metadata (see pages 87 and 88 of the book for details of implementing this)

### Random Forest

In [138]:
import org.apache.spark.ml.classification.RandomForestClassifier

import org.apache.spark.ml.classification.RandomForestClassifier

In [185]:
val classifierRF = new org.apache.spark.ml.classification.RandomForestClassifier().
                setSeed(Random.nextLong()).
                setLabelCol("Cover_Type").
                setFeaturesCol("featureVector").
                setPredictionCol("prediction").setImpurity("gini").setMaxBins(300).setMaxDepth(20).setMinInfoGain(0.0).setNumTrees(20)

classifierRF: RandomForestClassifier = rfc_b624caeac2a0

In [186]:
val fittedRF = classifierRF.fit(assembledTrainData)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

collectAsMap at RandomForest.scala:557

4 / 4

first at RandomForestClassifier.scala:145

1 / 1

fittedRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_b624caeac2a0) with 20 trees

In [189]:
val predsRF = fittedRF.transform(assembledTrainData)

predsRF: DataFrame = [Elevation: int, Aspect: int ... 57 more fields]

In [190]:
predsRF.select("Cover_Type","prediction","probability").show(truncate=false)

show at cmd189.sc:1

1 / 1

+----------+----------+------------------------------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                                         |
+----------+----------+------------------------------------------------------------------------------------------------------------------------------------+
|6.0       |6.0       |[0.0,0.0,0.01286902218169958,0.39028278938095196,0.012549260305912111,0.001780799740974583,0.5825181283904618,0.0]                  |
|6.0       |6.0       |[0.0,0.0,0.015496405244834838,0.20162844460153445,0.16445566359827607,0.0,0.6184194865553546,0.0]                                   |
|6.0       |6.0       |[0.0,0.0,0.005804132913535674,0.28195511710652077,0.010995917544413676,0.0,0.7012448324355299,0.0]                                  |
|6.0       |6.0       |[0.0,0.0,0.017293458271925147,0.277

In [191]:
predsRF.select("Cover_Type","prediction").groupBy("Cover_Type").pivot("prediction",(1 to 7)).count().na.fill(0).sort("cover_type").show()

show at cmd190.sc:1

4 / 4

show at cmd190.sc:1

200 / 200

show at cmd190.sc:1

200 / 200

+----------+------+------+-----+----+----+-----+-----+
|Cover_Type|     1|     2|    3|   4|   5|    6|    7|
+----------+------+------+-----+----+----+-----+-----+
|       1.0|156261| 33723|   26|   0|   7|    4|  545|
|       2.0| 19654|233639| 1085|   4|  95|  295|   71|
|       3.0|     1|  2042|29661|  42|   0|  488|    0|
|       4.0|     0|     0|  384|2053|   0|   27|    0|
|       5.0|   343|  6028|  166|   0|1971|   18|    0|
|       6.0|     4|  2580| 2948|  27|   0|10043|    0|
|       7.0|  3776|    66|    0|   0|   0|    0|14598|
+----------+------+------+-----+----+----+-----+-----+



Note: Without setting the defaults for impurity,infogain etc the confusion matrix shows no predictions for 4,5,6,7. The decisiontreeclassifier showed few predictions in this area but certainly more than zero. With them set to the values found for the descisiontree classifier then the results look much better

In [192]:
val evalRF = new org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator().
                        setLabelCol("Cover_Type").setPredictionCol("prediction").setMetricName("accuracy")


evalRF: MulticlassClassificationEvaluator = mcEval_4d9879dae3cf

In [193]:
evalRF.evaluate(predsRF)

map at MulticlassMetrics.scala:45

4 / 4

collectAsMap at MulticlassMetrics.scala:47

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

countByValue at MulticlassMetrics.scala:42

4 / 4

res192: Double = 0.8575615822451811

Note: Out of the box the random forest classifier resulted in an accuracy of 67%!!! 84% was acheived with the tuning due to setting the deafult hyperparameters to that of the decision tree optimal values.

In [194]:
fittedRF.explainParams

res193: String = """cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. (default: false)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name (default: features, current: featureVector)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini, current: gini)
labelCol: label column name (default: label, current: Cover_Type)
maxBins: Max number of bins for discr

Try some hyper parameter tuning. Create the pipeline; create the ParamGridBuilder; addGrid to set parameters of interest; create evaluator; use a TrainValidationSplit instance to run the modell fititng on the grid. This will return a list of fitted Pipelines with the best pipeline being accessible from the .stages.last. The model then needs to be extracted.

In [195]:
val rfPipeline = new org.apache.spark.ml.Pipeline()
rfPipeline.setStages(Array(assembler,classifierRF))


rfPipeline: Pipeline = pipeline_53a3eacdd8b7
res194_1: Pipeline = pipeline_53a3eacdd8b7

Try changing the number of trees only. 

In [196]:
val paramGridRF = new ParamGridBuilder().
                addGrid(classifierRF.numTrees, Seq(20,30)).
                build()

paramGridRF: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	rfc_b624caeac2a0-numTrees: 20
},
  {
	rfc_b624caeac2a0-numTrees: 30
}
)

In [181]:
evalRF.explainParams

res180: String = """labelCol: label column name (default: label, current: Cover_Type)
metricName: metric name in evaluation (f1|weightedPrecision|weightedRecall|accuracy) (default: f1, current: accuracy)
predictionCol: prediction column name (default: prediction, current: prediction)"""

In [197]:
val validatorRF = new TrainValidationSplit().
                        setSeed(Random.nextLong()).
                        setEstimator(rfPipeline).
                        setEvaluator(evalRF).setEstimatorParamMaps(paramGridRF).
                        setTrainRatio(0.9)

validatorRF: TrainValidationSplit = tvs_0b829f72264e

In [198]:
val validatorModel = validatorRF.fit(trainData)

take at Classifier.scala:111

4 / 4

take at Classifier.scala:111

1 / 1

take at DecisionTreeMetadata.scala:112

...

collectAsMap at RandomForest.scala:557

4 / 4

first at RandomForestClassifier.scala:145

1 / 1

validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_0b829f72264e

In [200]:
validatorModel.validationMetrics.zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)

res199: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.838913889100755,
    {
	rfc_b624caeac2a0-numTrees: 20
}
  ),
  (
    0.8375409961101365,
    {
	rfc_b624caeac2a0-numTrees: 30
}
  )
)

The above tuning doesn't improve the random forers performance even if replicating the decision tree. 

In [201]:
validatorModel.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res200: org.apache.spark.ml.param.ParamMap = {
	rfc_b624caeac2a0-cacheNodeIds: false,
	rfc_b624caeac2a0-checkpointInterval: 10,
	rfc_b624caeac2a0-featureSubsetStrategy: auto,
	rfc_b624caeac2a0-featuresCol: featureVector,
	rfc_b624caeac2a0-impurity: gini,
	rfc_b624caeac2a0-labelCol: Cover_Type,
	rfc_b624caeac2a0-maxBins: 300,
	rfc_b624caeac2a0-maxDepth: 20,
	rfc_b624caeac2a0-maxMemoryInMB: 256,
	rfc_b624caeac2a0-minInfoGain: 0.0,
	rfc_b624caeac2a0-minInstancesPerNode: 1,
	rfc_b624caeac2a0-numTrees: 20,
	rfc_b624caeac2a0-predictionCol: prediction,
	rfc_b624caeac2a0-probabilityCol: probability,
	rfc_b624caeac2a0-rawPredictionCol: rawPrediction,
	rfc_b624caeac2a0-seed: -4664685616551187652,
	rfc_b624caeac2a0-subsamplingRate: 1.0
}

Can't get the random forest to outperfrom the decision tree! Stackoverflow suggests that if the subtrees are stable then the rf and the decision trees perform the same. 


Might be the case that one hot encoding is reducing the performance. Will try the same experiments with usnig indexedVector representation of the categorical variables. 

In [287]:
import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.Vector // this required so that spark.ml.Vector type is imported. Fall back si the scala vector which takes a type

import org.apache.spark.sql.functions._

import org.apache.spark.ml.linalg.Vector // this required so that spark.ml.Vector type is imported. Fall back si the scala vector which takes a type

The code below initially failed because the vector class I was importing was spark.mllib.linalg.Vector not spark.ml.linalg.Vector

The purpose of this method is to overwrite the n one-hot encoded columns with the a single vector column to represent the one hot encoded values in a single field. Vector Indexer, apparently, attaches certain meta-data that avoids this column being treated as a numeric column as in the case of one hot encoding.


In [379]:
def unencodeOneHot(soilColCount: Int)(data: DataFrame) ={
    val wildernessCols=(0 until 4).map(i=>s"Wilderness_Area_$i").toArray
    
    val wildernessAssembler = new VectorAssembler().
                    setInputCols(wildernessCols).
                    setOutputCol("wilderness")
    
    // This UDF used to convert multiple columns of one-hot encoded data to a single vector column
    val unhotUDF = udf((vec: Vector) => vec.toArray.indexOf(1.0).toDouble)
    
    val withWilderness =   wildernessAssembler.transform(data).
                            drop(wildernessCols:_*).
                            withColumn("wilderness",unhotUDF($"wilderness"))
    

   val soilCols = (0 until soilColCount).map(i => s"Soil_Type_$i").toArray
    
    val soilAssembler = new VectorAssembler().
            setInputCols(soilCols).
            setOutputCol("soil")
    
   soilAssembler.transform(withWilderness).
            drop(soilCols:_*).
            withColumn("soil", unhotUDF($"soil")) 
}

defined function unencodeOneHot

In [383]:
val unencodedTrainData = unencodeOneHot(40)(trainData)
//unencodedTrainData.columns.foreach(println)
unencodedTrainData.count()
trainData.count()

count at cmd382.sc:3

4 / 4

count at cmd382.sc:3

1 / 1

count at cmd382.sc:4

4 / 4

count at cmd382.sc:4

1 / 1

unencodedTrainData: DataFrame = [Elevation: int, Aspect: int ... 11 more fields]
res382_1: Long = 522896L
res382_2: Long = 522896L

In [347]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class onehot(Wilderness_Area_0:Double,Wilderness_Area_1:Double,Wilderness_Area_2:Double,Wilderness_Area_3:Double, Soil_Type_0:Double,Soil_Type_1:Double)


defined class onehot

In [356]:
val test = Seq(onehot(0,1,0,0,1,0),onehot(0,0,1,0,0,1),onehot(1,0,0,0,1,0),onehot(0,0,0,1,0,1)).toDF()



test: DataFrame = [Wilderness_Area_0: double, Wilderness_Area_1: double ... 4 more fields]

In [357]:
test.show()

+-----------------+-----------------+-----------------+-----------------+-----------+-----------+
|Wilderness_Area_0|Wilderness_Area_1|Wilderness_Area_2|Wilderness_Area_3|Soil_Type_0|Soil_Type_1|
+-----------------+-----------------+-----------------+-----------------+-----------+-----------+
|              0.0|              1.0|              0.0|              0.0|        1.0|        0.0|
|              0.0|              0.0|              1.0|              0.0|        0.0|        1.0|
|              1.0|              0.0|              0.0|              0.0|        1.0|        0.0|
|              0.0|              0.0|              0.0|              1.0|        0.0|        1.0|
+-----------------+-----------------+-----------------+-----------------+-----------+-----------+



In [358]:
test.printSchema

root
 |-- Wilderness_Area_0: double (nullable = false)
 |-- Wilderness_Area_1: double (nullable = false)
 |-- Wilderness_Area_2: double (nullable = false)
 |-- Wilderness_Area_3: double (nullable = false)
 |-- Soil_Type_0: double (nullable = false)
 |-- Soil_Type_1: double (nullable = false)



In [364]:
unencodeOneHot(2)(test).show()

+----------+----+
|wilderness|soil|
+----------+----+
|       1.0| 0.0|
|       2.0| 1.0|
|       0.0| 0.0|
|       3.0| 1.0|
+----------+----+



Recreate the data with the indexed vector column repalcing the one hot encoded. Indexer (defined below) is a transformation and is added to the pipeline to convert the existing data to a form the classifier can use.

In [311]:
import org.apache.spark.ml.feature.VectorIndexer

import org.apache.spark.ml.feature.VectorIndexer

In [389]:
//Indexer transformation to replace the columsn wiht a single column of type vector that captures the categorical values
val inputCols = unencodedTrainData.columns.filter(_ != "Cover_Type")

val assembler = new VectorAssembler().
                    setHandleInvalid("skip").
                    setInputCols(inputCols).
                    setOutputCol("featureVector")

/*val indexer = new VectorIndexer().
                setMaxCategories(40).
                setInputCol("featureVector").
                setOutputCol("indexedVector")*/

//change the classifier configuration to take the indexedVector (categorical values non-onehot encoded)
val classifierRF_cat = new DecisionTreeClassifier().
                    setSeed(Random.nextLong()).
                    setLabelCol("Cover_Type").
                    setFeaturesCol("featureVector").
                    setPredictionCol("prediction")

val pipeline_cat = new Pipeline().setStages(Array(assembler, classifierRF_cat))
//val pipeline_cat = new Pipeline().setStages(Array(assembler, indexer, classifierRF_cat))

inputCols: Array[String] = Array(
  "Elevation",
  "Aspect",
  "Slope",
  "Horizontal_Distance_To_Hydrology",
  "Vertical_Distance_To_Hydrology",
  "Horizontal_Distance_To_Roadways",
  "Hillshade_9am",
  "Hillshade_Noon",
  "Hillshade_3pm",
  "Horizontal_Distance_To_Fire_Points",
  "wilderness",
  "soil"
)
assembler: VectorAssembler = vecAssembler_64e26f5d5810
classifierRF_cat: DecisionTreeClassifier = dtc_7325f39b25bc
pipeline_cat: Pipeline = pipeline_2be9015d12ad

We now have a pipeline that we can use to run the parameter tuning.

In [390]:
val validatorRF_Cat = new TrainValidationSplit().
                        setSeed(Random.nextLong()).
                        setEstimator(pipeline_cat).
                        setEvaluator(evalRF).setEstimatorParamMaps(paramGridRF).
                        setTrainRatio(0.9)

validatorRF_Cat: TrainValidationSplit = tvs_dde077e9194f

In [396]:
val validatorModelRF_cat = validatorRF_Cat.fit(unencodedTrainData)

take at Classifier.scala:111

4 / 4

19/11/12 06:36:45 ERROR Executor: Exception in task 3.0 in stage 2780.0 (TID 16952)
org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$5429/1989943469: (struct<Wilderness_Area_0_double_vecAssembler_b489cee78e46:double,Wilderness_Area_1_double_vecAssembler_b489cee78e46:double,Wilderness_Area_2_double_vecAssembler_b489cee78e46:double,Wilderness_Area_3_double_vecAssembler_b489cee78e46:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$2.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution

scala.concurrent.Future$.apply(Future.scala:659)
org.apache.spark.ml.tuning.TrainValidationSplit.$anonfun$fit$4(TrainValidationSplit.scala:160)
scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
scala.collection.TraversableLike.map(TraversableLike.scala:238)
scala.collection.TraversableLike.map$(TraversableLike.scala:231)
scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:198)
org.apache.spark.ml.tuning.TrainValidationSplit.$anonfun$fit$1(TrainValidationSplit.scala:149)
org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:183)
scala.util.Try$.apply(Try.scala:213)
org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:183)
org.apache.spark.ml.tuning.TrainValidationSplit.fit(TrainValid

take at Classifier.scala:111

4 / 4

19/11/12 06:36:45 ERROR Executor: Exception in task 3.0 in stage 2782.0 (TID 16956)
org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$5429/1989943469: (struct<Wilderness_Area_0_double_vecAssembler_b489cee78e46:double,Wilderness_Area_1_double_vecAssembler_b489cee78e46:double,Wilderness_Area_2_double_vecAssembler_b489cee78e46:double,Wilderness_Area_3_double_vecAssembler_b489cee78e46:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$2.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution

scala.concurrent.Future$.apply(Future.scala:659)
org.apache.spark.ml.tuning.TrainValidationSplit.$anonfun$fit$4(TrainValidationSplit.scala:160)
scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
scala.collection.TraversableLike.map(TraversableLike.scala:238)
scala.collection.TraversableLike.map$(TraversableLike.scala:231)
scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:198)
org.apache.spark.ml.tuning.TrainValidationSplit.$anonfun$fit$1(TrainValidationSplit.scala:149)
org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:183)
scala.util.Try$.apply(Try.scala:213)
org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:183)
org.apache.spark.ml.tuning.TrainValidationSplit.fit(TrainValid

almond.Execute.withOutputHandler(Execute.scala:155)
almond.Execute.ammResult(Execute.scala:211)
almond.Execute.apply(Execute.scala:295)
almond.ScalaInterpreter.execute(ScalaInterpreter.scala:117)
almond.interpreter.InterpreterToIOInterpreter.$anonfun$execute$2(InterpreterToIOInterpreter.scala:69)
cats.effect.internals.IORunLoop$.cats$effect$internals$IORunLoop$$loop(IORunLoop.scala:87)
cats.effect.internals.IORunLoop$RestartCallback.signal(IORunLoop.scala:355)
cats.effect.internals.IORunLoop$RestartCallback.apply(IORunLoop.scala:376)
cats.effect.internals.IORunLoop$RestartCallback.apply(IORunLoop.scala:316)
cats.effect.internals.IOShift$Tick.run(IOShift.scala:36)
java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
java.lang.Thread.run(Thread.java:748)


: 

Get the model valuation metrics out of it

In [265]:
validatorModelRF_cat.validationMetrics.zip(validatorModelRF_cat.getEstimatorParamMaps).sortBy(-_._1)

res264: Array[(Double, org.apache.spark.ml.param.ParamMap)] = Array(
  (
    0.7039439593129259,
    {
	rfc_b624caeac2a0-numTrees: 20
}
  ),
  (
    0.7039439593129259,
    {
	rfc_b624caeac2a0-numTrees: 30
}
  )
)

In [261]:
validatorModelRF_cat.bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

res260: org.apache.spark.ml.param.ParamMap = {
	dtc_e68c7fc9cdf6-cacheNodeIds: false,
	dtc_e68c7fc9cdf6-checkpointInterval: 10,
	dtc_e68c7fc9cdf6-featuresCol: featureVector,
	dtc_e68c7fc9cdf6-impurity: gini,
	dtc_e68c7fc9cdf6-labelCol: Cover_Type,
	dtc_e68c7fc9cdf6-maxBins: 32,
	dtc_e68c7fc9cdf6-maxDepth: 5,
	dtc_e68c7fc9cdf6-maxMemoryInMB: 256,
	dtc_e68c7fc9cdf6-minInfoGain: 0.0,
	dtc_e68c7fc9cdf6-minInstancesPerNode: 1,
	dtc_e68c7fc9cdf6-predictionCol: prediction,
	dtc_e68c7fc9cdf6-probabilityCol: probability,
	dtc_e68c7fc9cdf6-rawPredictionCol: rawPrediction,
	dtc_e68c7fc9cdf6-seed: -2190394278285637533
}

In [ ]:
val paramGridRF = new ParamGridBuilder().
                addGrid(classifierRF.numTrees, Seq(20,30)).
                build()